Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [ ]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import random
from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [116]:
INSTANCES = [
    {"cities": pd.read_csv("cities/italy.csv",   header=None, names=["name", "lat", "lon"]), "dist_matrix": None, "name": "Italy"},
    {"cities": pd.read_csv("cities/china.csv",   header=None, names=["name", "lat", "lon"]), "dist_matrix": None, "name": "China"},
    {"cities": pd.read_csv("cities/russia.csv",  header=None, names=["name", "lat", "lon"]), "dist_matrix": None, "name": "Russia"},
    {"cities": pd.read_csv("cities/us.csv",      header=None, names=["name", "lat", "lon"]), "dist_matrix": None, "name": "US"},
    {"cities": pd.read_csv("cities/vanuatu.csv", header=None, names=["name", "lat", "lon"]), "dist_matrix": None, "name": "Vanuatu"}
]
for instance in INSTANCES:
    cities = instance["cities"]
    dist_matrix = np.zeros((len(cities), len(cities)))
    for c1, c2 in combinations(cities.itertuples(), 2):
        dist_matrix[c1.Index, c2.Index] = dist_matrix[c2.Index, c1.Index] = geodesic(
            (c1.lat, c1.lon), (c2.lat, c2.lon)
        ).km
    instance["dist_matrix"] = dist_matrix
    cities.head()

## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [117]:
def tsp_cost(instance, tsp):
    cities = instance["cities"]
    dist_matrix = instance["dist_matrix"]
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(cities)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += dist_matrix[c1, c2]
    return tot_cost

## Greedy Algorithm

In [118]:
def greedy(instance, print):
    cities = instance["cities"]
    dist_matrix = instance["dist_matrix"]
    visited = np.full(len(cities), False)
    dist = dist_matrix.copy()
    city = 0
    visited[city] = True
    tsp = list()
    tsp.append(int(city))
    while not np.all(visited):
        dist[:, city] = np.inf
        closest = np.argmin(dist[city])
        #logging.debug(
        #    f"step: {cities.at[city,'name']} -> {cities.at[closest,'name']} ({dist_matrix[city,closest]:.2f}km)"
        #)
        visited[closest] = True
        city = closest
        tsp.append(int(city))
    #logging.debug(
    #    f"step: {cities.at[tsp[-1],'name']} -> {cities.at[tsp[0],'name']} ({dist_matrix[tsp[-1],tsp[0]]:.2f}km)"
    #)
    tsp.append(tsp[0])

    if print:
        logging.info(f" result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(instance, tsp):.2f}km")
    return tsp

In [119]:
i=1
for instance in INSTANCES:
    logging.info(f" instance {i} ({instance["name"]}):")
    greedy(instance, True)
    i += 1 

INFO:root: instance 1 (Italy):
INFO:root: result: Found a path of 46 steps, total length 4436.03km
INFO:root: instance 2 (China):
INFO:root: result: Found a path of 726 steps, total length 63962.92km
INFO:root: instance 3 (Russia):
INFO:root: result: Found a path of 167 steps, total length 42334.16km
INFO:root: instance 4 (US):
INFO:root: result: Found a path of 326 steps, total length 48050.03km
INFO:root: instance 5 (Vanuatu):
INFO:root: result: Found a path of 8 steps, total length 1475.53km


# Evolutionary Algorithm

In [ ]:
def ea(instance, pop_size=100, generations=500, initial_pop_multiplier=2, mutation_rate=0.1, elitism_rate=0.05):
    # Dati di input: città e matrice delle distanze
    cities = instance["cities"]
    dist_matrix = instance["dist_matrix"]
    num_cities = len(cities)

    # Funzione per calcolare il costo di un percorso
    def path_cost(path):
        return tsp_cost(instance, path + [path[0]])

    # Selezione tramite torneo: seleziona il percorso con il costo minimo tra k individui
    def tournament_selection(population, k=5):
        tournament = random.sample(population, k)
        return min(tournament, key=path_cost)
    
    # Operatore di crossover: combina due genitori per produrre un figlio
    def order_crossover(parent1, parent2):
        start, end = sorted(random.sample(range(num_cities), 2))  # Segmento da copiare dal primo genitore
        child = [None] * num_cities
        child[start:end] = parent1[start:end]  # Copia il segmento nel figlio
        pos = end
        for city in parent2:  # Riempie le restanti città evitando duplicati
            if city not in child:
                if pos >= num_cities:
                    pos = 0
                child[pos] = city
                pos += 1
        return child
    
    # Operatore di mutazione: inverte un sottoinsieme casuale del percorso
    def inversion_mutation(tour):
        i, j = sorted(random.sample(range(num_cities), 2))
        tour[i:j] = reversed(tour[i:j])
        return tour

    # Inizializzazione della popolazione con percorsi casuali
    population = [random.sample(range(num_cities), num_cities) for _ in range(pop_size * initial_pop_multiplier)]
    best_solution = min(population, key=path_cost)  # Trova il percorso migliore iniziale
    best_cost = path_cost(best_solution)

    # Ciclo evolutivo principale
    for generation in range(1, generations + 1):
        new_population = []

        # Elitismo: mantiene una percentuale dei migliori individui
        elite_count = max(1, int(elitism_rate * pop_size))
        elite_individuals = sorted(population, key=path_cost)[:elite_count]
        new_population.extend(elite_individuals)

        # Generazione di nuovi individui attraverso crossover e mutazione
        while len(new_population) < pop_size:
            parent1 = tournament_selection(population, k=10)
            parent2 = tournament_selection(population, k=10)

            prob = 0.8 # Applica crossover con probabilità 80%
            if random.random() < prob: 
                child1 = order_crossover(parent1, parent2)
                child2 = order_crossover(parent2, parent1)
            else:  # In caso contrario, copia i genitori direttamente
                child1, child2 = parent1[:], parent2[:]

            # Applica mutazione con probabilità adattiva che decresce nel tempo
            adaptive_mutation_rate = max(0.01, mutation_rate - (mutation_rate * generation / generations))
            if random.random() < adaptive_mutation_rate:
                child1 = inversion_mutation(child1)
            if random.random() < adaptive_mutation_rate:
                child2 = inversion_mutation(child2)

            new_population.extend([child1, child2])

        # Aggiorna la popolazione con i nuovi individui
        population = new_population[:pop_size]

        # Aggiorna la migliore soluzione se è stato trovato un percorso con costo inferiore
        current_best = min(population, key=path_cost)
        current_cost = path_cost(current_best)

        if current_cost < best_cost:
            best_solution = current_best
            best_cost = current_cost
            # logging.info(f"Generation {generation}: New best cost = {best_cost:.2f} km")

    # Completa il percorso tornando alla città di partenza
    best_solution.append(best_solution[0])
    logging.info(f" result: Found a path of {len(best_solution) - 1} steps, total length {best_cost:.2f}km")

    return best_solution, best_cost


In [121]:
i=1
for instance in INSTANCES:
    logging.info(f" instance {i} ({instance["name"]}):")
    ea(instance)
    i += 1 

INFO:root: instance 1 (Italy):
INFO:root: result: Found a path of 46 steps, total length 4721.66km
INFO:root: instance 2 (China):
INFO:root: result: Found a path of 726 steps, total length 367689.23km
INFO:root: instance 3 (Russia):
INFO:root: result: Found a path of 167 steps, total length 65540.95km
INFO:root: instance 4 (US):
INFO:root: result: Found a path of 326 steps, total length 142000.39km
INFO:root: instance 5 (Vanuatu):
INFO:root: result: Found a path of 8 steps, total length 1345.54km
